# NanoVer ASE Client / Server Example

This notebook demonstrates how to set up a molecular simulation with ASE and combine it with NanoVer to run an interactive molecular dynamics (iMD) simulation. 
We then connect to the simulation from a client, so we can look at the data being produced and visualize it.


**Note**: This example demonstrates running iMD simulations using ASE via NanoVer. For tutorials that demonstrate the different ways to interface ASE with OpenMM via NanoVer, check out the tutorials in this directory prefixed with `ase_openmm`.

## Set up an ASE simulation

First, we set up an ASE simulation. We're going to do a simple lattice of copper particles under an EMT potential, but you can use [any supported calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html#module-ase.calculators)!

In [1]:
from ase import units
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [2]:
size = 2
# Set up a crystal
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol="Cu",
                          size=(size, size, size),
                          pbc=True)
# Describe the interatomic interactions with Effective Medium Theory
atoms.calc = EMT()
# Set the atomic momenta to 300 Kelvin. 
MaxwellBoltzmannDistribution(atoms, temperature_K=300)

Next we need to define the component of the molecular dynamics simulation that performs the numerical integration of the equations of motion. We're going to run Langevin dynamics at room temperature:

In [3]:
dyn = Langevin(atoms, timestep=1 * units.fs, temperature_K=300, friction=0.5)

Let's check the dynamics are working by running one simulation step and checking the potential energy.

**WARNING**: when accessing data via the `dyn` object, the values we retrieve are in the *units used internally by ASE*. *These units **differ** from those of NanoVer*, so bear this in mind when examining data this way!

In [4]:
dyn.run(steps = 1)
dyn.get_number_of_steps()

1

In [5]:
# Note that this is the energy in eV, not kJ mol-1 (the standard units of ASE are different to those of OpenMM and NanoVer)
dyn.atoms.get_potential_energy()

np.float64(-0.1804703825507339)

Now let's import the `ASESimulation` class: this allows us to pass our ASE molecular dynamics simulation to NanoVer to run an iMD simulation. We can define our `ASESimulation` from the dynamics object we created above, as follows:

In [6]:
from nanover.ase import ASESimulation

ase_sim = ASESimulation.from_ase_dynamics(dyn)
ase_sim.frame_interval = 1  # report every simulation step to clients

Let's run a couple of steps to check that everything is still working. 

**NOTE**: the commands below still run the ASE MD simulation directly, (i.e. *NOT* using NanoVer and thus not running an iMD simulation), and the data is still being accessed via the dynamics object defined previously (i.e. `ase_sim.dynamics = dyn`, see above). 

In [7]:
ase_sim.dynamics.run(10)
ase_sim.dynamics.nsteps

11

In [8]:
ase_sim.dynamics.atoms.get_potential_energy()

np.float64(-0.059623275783556196)

Great! Everything's working with the `ASESimulation`. Let's turn our attention to setting up the server and running an iMD simulation.

## Set up the NanoVer iMD server

In NanoVer we use the `OmniRunner` class to serve iMD simulations.

In [9]:
from nanover.omni import OmniRunner

We set up an iMD simulation by passing the `ASESimulation` defined above to an `OmniRunner`: we set `port=0` to let the operating system pick a free port for us.

In [10]:
imd_runner = OmniRunner.with_basic_server(ase_sim, port=0, name="ase_nanover_server") #Try using Shift+TAB+TAB to see what you can set up here.

**Note**: Be careful if you run the above cell multiple times without running `imd_runner.close()` (see below), as you will start multiple servers, which may be discovered if you use autoconnect. You can guard against this by swapping the cell above with:

```python
try:
    imd_runner.close()
except NameError: # If the server hasn't been defined yet, there will be an error
    pass
imd_runner = OmniRunner.with_basic_server(ase_sim, port=0)
```

Our server is now running and discoverable! We can see where it's running by running the following cell:

In [11]:
imd_runner.print_basic_info()

Serving "ase_nanover_server" (ws://localhost:55031), discoverable on all interfaces on port 54545
Available simulations:
0: "Unnamed ASE OpenMM Simulation"


The `[::]` means it is running on all available addresses, e.g. your WiFi and cabled access, and it's found an available port

Now, let's start the ASE simulation

In [12]:
imd_runner.load(0)

In [13]:
# print the time to check dynamics is running
print(f'Simulation time: {ase_sim.dynamics.get_time()}, ({ase_sim.dynamics.nsteps} steps)')

Simulation time: 1.080496426731047, (11 steps)


Ok, it's working, so let's leave it running dynamics in a background thread. This is what happens by default when you call `.next()`

That's it, your simulation is running and interactive! If you connect to it from iMD-VR, you'll see something like:

![NanoVer ASE EMT](./images/nanover_ase_emt.png)

You can also visualize it live from within a Jupyter notebook, (check out this [example with NGLView](../basics/nanover_nglview.ipynb)).

## Start an IMD client

Let's have a look at the data that's being produced here, by connecting a python client to the server running the simulation.

In [14]:
from nanover.websocket import NanoverImdClient
client = NanoverImdClient.from_runner(imd_runner)

Let's grab one of the frames and examine some of its values

In [15]:
frame = client.wait_until_first_frame()

In [16]:
frame

<FrameData of bond.pairs[0, 2], chain.count, chain.names[1], energy.kinetic, frame.index, particle.count, particle.elements[32], particle.names[32], particle.positions[32, 3], particle.residues[32], residue.chains[1], residue.count, residue.ids[1], residue.names[1], server.timestamp, system.box.vectors[3, 3], system.simulation.counter>

In [17]:
# NBVAL_SKIP
# Print the potential energy, here in kJ mol-1 as we are accessing it via NanoVer
frame.potential_energy # use TAB to see what's available!

-3.991258561786523

See here that as we have accessed the potential energy via the frame output by NanoVer, so it is given in NanoVer's standard unit of energy, kJ/mol.

Now, let's visualize the frame directly with ASE. First we convert the frame back into ASE atoms, and view it. The first frame received by a client always has the topology information

In [18]:
from nanover.ase.converter import frame_data_to_ase
atoms = frame_data_to_ase(client.current_frame, topology=True, positions=False)

We can run this to update the latest positions from the frame (note that NanoVer uses **nanometers** for positions ([the same as OpenMM](http://docs.openmm.org/6.2.0/userguide/theory.html#units)), so we need to convert back to Angstrom to visualise the system using ASE.)

In [19]:
import numpy as np
# set the positions to match the latest frame data
atoms.set_positions(np.array(client.current_frame.particle_positions) * 10)

ASE has a cool little 2D visualizer

In [20]:
from ase.visualize import view
view(atoms)

<Popen: returncode: None args: ['C:\\Users\\ragzo\\anaconda3\\envs\\nanover-...>

And a 3D visualizer! (Check your browser)

In [21]:
view(atoms, viewer='x3d')

# Gracefully terminate!!

It is very important to close your servers, otherwise they'll get in the way by blocking ports, and your clients may autoconnect to the wrong server! We close the server calling `.close()`

In [22]:
imd_runner.close()

Note that outside of a notebook, you can use `with` statements to let NanoVer clean up after itself:

In [23]:
with OmniRunner.with_basic_server(ASESimulation.from_ase_dynamics(dyn), port=0) as imd_runner:
    imd_runner.next()

# Next Steps

This notebook demonstrated how to use NanoVer to run iMD simulations using ASE for a toy system. There's plenty more that can be done!

* Run bigger molecular mechanics simulations using OpenMM. The [nanotube](./ase_openmm_nanotube.ipynb) and [neuraminidase](./ase_openmm_neuraminidase.ipynb) examples show how to set up simulations that combine OpenMM with ASE.
* Use the functionality of ASE change the parameters of an OpenMM simulation on the fly. See the [graphene](./ase_openmm_graphene.ipynb) example.
* To understand what's going on under the hood, check out the [NanoVer frame explained](../fundamentals/frame.ipynb) example. 